In [2]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

# 표를 불러오기 위한 라이브러리
from html_table_parser import parser_functions as parser
import collections

from IPython.display import clear_output

collections.Callable = collections.abc.Callable
# 403 에러 방지
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [4]:
cats = {
    "반팔": "001001",
    "맨투맨": "001005",
    "후드집업": "002022",
    "숏패딩": "002012",
    "데님팬츠": "003002",
    "코튼팬츠": "003007",
}

for cat in cats:
    if cat == "반팔" or cat =="맨투맨" or cat == "후드집업" or cat == "숏패딩":
        continue

    selected_cat = cat
    # print(selected_cat)

    # csv 파일 읽기
    dir = "url_list/"
    file_name = dir + selected_cat + "_url_list.csv"
    # file_name = selected_cat + "_url_list.csv"
    data = pd.read_csv(file_name, names=["url"])
    # urls = data["url"].values
    # print(urls.shape, urls)

    url_df = pd.DataFrame(data, columns=["url"])
    url_list = url_df.values.tolist()

    size_review_df = pd.DataFrame()

    for i in range(len(url_list) - 1):
        try:
            url = url_list[i + 1][0]

            """if i % 10 == 0:
                print(i, " / ", len(url_list), url)"""

            page = requests.get(url, headers=headers)
            soup = bs(page.text, "lxml")
            table = soup.find_all(attrs={"class": "table_th_grey"})

            p = parser.make2d(table[0])
            p[0][0] = "사이즈"
            size_df = pd.DataFrame(p[3:], columns=p[0])
            # size_df.insert(0, 'url', url)
            # print(size_df)

            review_list = []
            page = requests.get(url, headers=headers)
            soup = bs(page.text, "lxml")
            # print(review)

            j = 0
            while j >= 0:
                try:
                    review = soup.find_all(attrs={"class": "size_content"})[
                        j
                    ].get_text()
                    review_list.append(review.split())
                    j += 1
                except:
                    break

            review_list2 = []
            for rv in review_list:
                temp = []
                for k in range(len(rv)):
                    if k == 1:
                        if rv[k][1:] == "남자":
                            temp.append(1)
                        elif rv[k][1:] == "여자":
                            temp.append(0)
                        else:
                            temp.append(-1)
                        # temp.append(rv[k][1:])
                    elif k == 2:
                        temp.append(rv[k].split("/")[0][:-2])
                        temp.append(rv[k].split("/")[1][:-3])
                    elif k == 0 or k == 3 or k == 6 or k == 7:
                        continue
                    else:
                        temp.append(rv[k])
                review_list2.append(temp)
            # print(review_list2)

            col_name = ["성별", "키", "몸무게", "후기", "사이즈"]
            review_df = pd.DataFrame(review_list2, columns=col_name)

            # print(review_df)

            temp_df = pd.merge(
                left=size_df,
                right=review_df,
                how="inner",
                left_on="사이즈",
                right_on="사이즈",
            )
            # temp_df.sort_values(["사이즈", "후기"], ascending=[True, True])

            size_review_df = pd.concat([size_review_df, temp_df])

            # print(temp_df)

            clear_output(wait=True)
            print(selected_cat, i, " / ", len(url_list), size_review_df.shape)
        except:
            continue

    #size_review_df = size_review_df[["총장", "어깨너비", "가슴단면", "소매길이", "성별", "키", "몸무게", "후기"]].dropna(axis=0)

    dir = "datasets/"
    file_name = dir + selected_cat + "_data.csv"
    # file_name = selected_cat + "_data.csv"
    size_review_df.to_csv(file_name, index=False)

코튼팬츠 4852  /  8911 (29199, 14)
